**Import Some Libraries**

In [265]:
import math as m
import numpy as np
from tabulate import tabulate

**Function Definitions**

In [266]:
def sqrt(number): # yeah, I am extremely lazy. What of it!?
    return m.sqrt(number)

def find_a(fc,fy,As,b): # seems this is not in ACI 318-19 directly, but this is analysis per Whitney stress block.
    a = (As*fy)/(0.85*fc*b)
    return a 

def find_beta1(fc): # from ACI 318-19 Table 22.2.2.4.3
    if fc <= 4:
        beta1 = 0.85
    elif fc >= 8:
        beta1 = 0.65
    else:
        beta1 = 0.85 - .05*(fc-4)
    return beta1

def find_c(a, beta1): # this is the depth to the neutral axis from using eqn 22.2.2.3 assuming "a" is known 
    c = a/beta1         
    return c

def find_Mn(fc,fy,As,b,d): # Whitney stress block approach to finding nominal bending moment capacity
    a = find_a(fc,fy,As,b)
    Mn = As*fy*(d-a/2)
    return Mn

def find_rho(As,b,d):   # ACI 318-19 2.2
    rho = As/(b*d)
    return rho

def find_ey(fy,Es): # ACI 318-19 20.2.2.1
    if fy == 60:
        ey = 0.002
    else:
        ey = fy/Es
    return ey

def find_et(d,c, eu=.003): # from using similar triangle relationship between c/eu = (d-c)/es
    et = eu*(d - c)/c                              
    return et

def find_Ec(fc,wc=None): # (19.2.2.1.a) or (19.2.2.1.b) # wc is density of concrete in lb/ft^3, Es is calculated in ksi for unit consistency. 
    if wc == None: # If wc not given, then normal weight concrete assumed (default function behavior)
        return 57000*sqrt(fc*1000)/1000 # use (19.2.2.1.b) for normal weight concrete
    if 60 < wc < 160:
        return wc**(1.5)*33*sqrt(fc*1000)/1000 # if wc is given, then (19.2.2.1.a) is used
    if wc < 60:
        raise Exception(f"{wc} lbs/cuft concrete is not dense enough (19.2.2.1)") # also watch depth/span ration per 7.3.1.1 and/or 9.3.1.1
    if wc > 160:
        raise Exception(f"{wc} lbs/cuft concrete is too dense (19.2.2.1)")

### use this function if lightweight concrete is being used. Not used by default
def find_lambda_conc(wc): # Table 21.2.4.1 (a)
    if wc <= 100:
        lam_c = 0.75
    if wc > 100: # Table 21.2.4.1 (a) eqn (b) and eqn (c) are covered here. For wc > 100, we use eqn (b), then...
        lam_c = 0.0075*wc
        if lam_c > 1.0: # ... if lambda_conc is greater than 1 as a result, we set lambda_conc equal to 1.0
            lam_c = 1.0
    return lam_c
    

def find_rho_max(fc,fy,beta1,ey, eu=.003):   # FIND THE REFERENCE!
    rho_max = 0.85*beta1*fc/fy*(eu)/(eu+ey+.003)
    return rho_max

def find_rho_bal(fc,fy,beta1,ey, eu=.003):   # FIND THE REFERENCE!
    rho_bal = 0.85*beta1*fc/fy*(eu)/(eu+ey)
    return rho_bal

def find_rho_min(fc,fy):    # FIND THE REFERENCE!
    rho_min = 3*sqrt(fc*1000)/fy/1000
    if rho_min > 200/(fy*1000):
        rho_min = 200/(fy*1000)
    return rho_min

def find_phi(rho,rho_max,rho_bal,et,ey): # ,et = 0,ey = 0):  # Table 21.2.2 assuming no spiral reinforcement (note, since this is non-prestressed, ey = ety in this formula)
    if rho <= rho_max: # 21.2.2 eqn (b)
        phi = 0.9
    elif rho_max < rho < rho_bal:      # 21.2.2 eqn (d), note if et, and ey are not given
        phi = .65 + .25*(et-ey)/.003
    else:                                # 21.2.2 eqn (f)
        phi = 0.65
    return phi

def comment_on_rho(rho,rho_min,rho_max): # FIND THE REFERENCE!
    if rho < rho_min:
        print("You need to add more rebar!")
        print()
    elif rho > rho_max:
        print("You should remove some rebar or increase your beam or slab depth (𝜌 is greater than 𝜌_max)!")
        print()
    else:
        print("Reinforcement ratio, 𝜌, looks okay (𝜌 is greater than 𝜌_min and  \n (𝜌 is less or equal to 𝜌_max).")
        print()

def comment_on_phi(phi):    # FIND THE REFERENCE!
    if phi == 0.9:
        print(f"We're in the tension-controlled zone, which is good... phi = {phi:.2f} (for flexure).")
        print()
    elif 0.65 < phi < 0.9:
        print(f"Hmmm, we're in the transition zone... Check your design. phi = {phi:.2f}.")
        print()
    else:
        print(f"Yikes! We're over-reinforced! Sudden failure is possible if overloaded. \n ... Actually... please fix it. Don't do it. \n Please! phi = {phi:.2f} (Very bad phi for flexure)")
        print()

def comment_on_h(d,h):  # this is just common sense. Reinforcement needs to be within "h" of the member
    if h < d:
        print("Check your dimensions, d cannot be larger than h! We're not in the Twilight Zone...")
        print()
    if h < d + 1.5:
        print("You might want to increase h to ensure proper concrete cover.")
        print()

def comment_on_lambda_conc(lambda_conc):
    if lambda_conc == 1:
        print("... Assuming normal weight concrete, so λ = 1 (19.2.4.3)")
    if lambda_conc != 1:
        print(f"... Assuming a concrete with λ = {lambda_conc} (19.2.4)")

def find_phi_Mn(phi,Mn): # honestly not sure why I did this... let's do this without a function
    phi_Mn = phi*Mn
    return phi_Mn



debug space

**Calculate Shear Capacity Values**

In [267]:
#Shear is complicated!

def find_Av_min(fc,fy,b): # ACI 318-19 Table 9.6.3.4 for nonprestressed members
    Av_min = .75*b/sqrt(fc*1000)/fy/1000
    if Av_min < 50*b/fy/1000:
        Av_min = 50*b/fy/1000   
    return Av_min

def find_lambda_size(d): # ACI 318-19 22.5.5.1.3
    if d <= 10:
        lambda_size = 1
    else:
        lambda_size = sqrt(2/(1+d/10))
    return lambda_size

def find_Vc(fc,b,d,h,As,Av,Av_min,member_type,lambda_size,lambda_conc): # ACI 318-19 7.6.3.1 or 9.6.3 and 22.5.5 (assuming no axial load to start)
    Vc = 0.0
    if member_type == "slab":
        if Av >= Av_min:
            print("Slab with Av >= Av_min (We have enough stirrups).")
            # 22.5.5.1 (a) chosen if Av >= Av_min
            Vc = 2*lambda_conc*sqrt(fc*1000)*b*d/1000                                     
            if Vc < 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000:
                # 22.5.5.1 (b) chosen if it is larger than (a)
                Vc = 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000                
        else:
            print("This is a slab with Av < Av_min (No stirrups or not enough stirrups).")
            # 22.5.5.1 (c) chosen if Av < Av_min                
            Vc = 8*lambda_size*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000   
    
    elif member_type == "beam":
        if Av >= Av_min:
            print("Beam with Av >= Av_min (We have enough stirrups).")
            Vc = 2*lambda_conc*sqrt(fc*1000)*b*d/1000                           # 22.5.5.1 (a)
            if Vc < 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000:
                Vc = 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000       # 22.5.5.1 (b)
        else: # Av < Av_min
            print("This is a beam with Av < Av_min (No stirrups or not enough stirrups).")
            if h <= 10:
                print("... but h <= 10 inches, so we don't have to meet Av_min requirements per Table 9.6.3.1 to be able to use 22.5.5.1 (c).")    
                # use 22.5.5.1 (c) assuming rectangular beam with Av < Av_min and h <= 10 inches per Table 9.6.3.1                
                Vc = 8*lambda_size*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000
            else: # h > 10
                print("... and h > 10 inches, so we have to meet Av_min requirements unless we have enough Vc for Vu at critical section per 9.6.3 AND 22.5.5.1 (c).")
                Vc = 8*lambda_size*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000
                # 9.6.3.1 limits shear strength for beams deeper than 10 inches
                if Vc > lambda_conc*sqrt(fc*1000)*b*d/1000:
                    Vc = lambda_conc*sqrt(fc*1000)*b*d/1000
                    print(" ... Might need some stirrups... 9.6.3.1 is limiting our shear strength!")
    print()
    return Vc

def adjust_Vc_with_Nu(Vc,fc,b,d,h,Nu): # ACI 318-19 22.5.5.1 and 22.5.5.1.2
    if Nu == 0:
        print("No axial load assumed, so no adjustment to Vc was done using Nu.")
        print()
    elif Nu != 0 and Nu/(6*b*h) <= .05*fc:
        Vc = Vc + Nu/6/(b*h)*b*d
        print(f"Axial load is present. Adjusting Vc using Nu since Nu/(6*b*h) = {Nu/(6*b*h)} is lower than 0.05fc = {.05*fc} (See 22.5.5.1.2).")
        if Nu < 0:
            print("Interesting and unusual... Nu is negative, so there appears to be tension involved.\n  ...Anyways, that is going to hurt our shear strength a bit or a lot")
        print()
    else:
        Vc = Vc + .05*fc*b*d
        print("Axial load is present. Adjusting Vc by adding .05*fc*b*d since Nu is too high. (See 22.5.5.1.2)")
        print()
        
    return Vc

def check_max_Vc(Vc,fc,b,d,member_type,lambda_conc): # ACI 318-19
    if Vc > 5*lambda_conc*sqrt(fc*1000)*b*d/1000:  # ACI 318-19 22.5.5.1.1
        Vc = 5*lambda_conc*sqrt(fc*1000)*b*d/1000
        print(f"Vc is being capped by 22.5.5.1.1... The {member_type} might need to have a bigger cross-section!")
        print()
    return Vc  

**Input Values**

In [268]:
# Commonly not changed values
Es = 29000                  # ksi # ACI 318-19 20.2.2.2

#Semi-Common inputs to change
fy = 60                     # ksi # Using grade 60 steel for the most part (fy = 60 ksi)
b = 12                      # inches # Not often changed for slabs and b=12 normally, as either E-width or effective b-width is used and loading is usual analyzed with *b = 12"* (per foot of width)
# wc = 143.9596             # note, only use this if concrete is known or if lightweight concrete per 19.2.4

#Common inputs to change
fc = 5                      # ksi
h = 8                       # in
d = 6.19                    # in
As = .44*4                  # in^2
Av = 0                      # in^2 # this is stirrup As per space, s. Do not forget to include other leg of stirrup if, for example, U-stirrup
# note: should include calculations for spacing of stirrups required at some point
member_type = "beam"        # beam or slab # Note that sometimes if b-width is small, "beam" is the more conservative assumption especially regarding Vc
lambda_conc = 1             # λ = 1 for normal weight (19.2.4.3), .75 for lightweight (19.2.4.2) Note: if light weight concrete, it may be good to make a function to find λ (lambda_conc)
Nu = 0                      # kips, positive for compression, negative for tension


**Print Input Values**

In [269]:

print("___GIVEN___")
print(f"fy = {fy} ksi")
print(f"Es = {Es} ksi")
print(f"fc = {fc} ksi")
print(f"b = {b} in")
print(f"h = {h} in")
print(f"d = {d} in")
print(f"As = {As} in^2")
print(f"Av = {Av} in^2")
print(f"member_type = {member_type}")
print(f"𝜆c = {lambda_conc} (𝜆c = 1 if normal weight concrete)")
print(f"Nu = {Nu} kips")
print()

# Find values
beta1 = find_beta1(fc)              
lambda_size = find_lambda_size(d)   # from ACI 318-19 22.5.5.1.3
Av_min = find_Av_min(fc,fy,b)       # ACI 318-19 Table 9.6.3.4 for nonprestressed members 
a = find_a(fc,fy,As,b)            
c = find_c(a, beta1)
ey = find_ey(fy,Es)

rho = find_rho(As,b,d)
rho_min = find_rho_min(fc,fy)
rho_max = find_rho_max(fc,fy,beta1,ey)
rho_bal = find_rho_bal(fc,fy,beta1,ey)

print("___QUICK CHECKS___")
print(f"rho = {rho:.4f} <-- should be less than rho_max for to be tension controlled if new design,\n...or, if reviewing an old design, this should be at least less than rho_bal)")
print(f"rho_max = {rho_max:.4f}")
print(f"rho_bal = {rho_bal:.4f}")

Mn = find_Mn(fc,fy,As,b,d)
et = find_et(d,c)
print(f"et = {et:.5f}")
phi = find_phi(rho,rho_max,rho_bal,et,ey)
print(f"phi = {phi:.2f}")
phi_Mn = phi*Mn



___GIVEN___
fy = 60 ksi
Es = 29000 ksi
fc = 5 ksi
b = 12 in
h = 8 in
d = 6.19 in
As = 1.76 in^2
Av = 0 in^2
member_type = beam
𝜆c = 1 (𝜆c = 1 if normal weight concrete)
Nu = 0 kips

___QUICK CHECKS___
rho = 0.0237 <-- should be less than rho_max for to be tension controlled if new design,
...or, if reviewing an old design, this should be at least less than rho_bal)
rho_max = 0.0212
rho_bal = 0.0340
et = 0.00417
phi = 0.83


**Output Commentary and Do Some Shear Calcs**

In [270]:

print("___NOTES AND CHECKS___")
# Shear, Vc, is now wildly complicated to calculate, so checks are done during calculations for now, 
# which is why they are in this "checks" section.
print(f"Calculations being done through use of ACI 318 - 19 ... \n ... All code references are per this Code. \n")
Vc = find_Vc(fc,b,d,h,As,Av,Av_min,member_type,lambda_size,lambda_conc)
Vc = adjust_Vc_with_Nu(Vc,fc,b,d,h,Nu)
Vc = check_max_Vc(Vc,fc,b,d,member_type,lambda_conc)
phi_Vc = 0.75*Vc
comment_on_h(d,h)
comment_on_rho(rho,rho_min,rho_max)
comment_on_phi(phi)


___NOTES AND CHECKS___
Calculations being done through use of ACI 318 - 19 ... 
 ... All code references are per this Code. 

This is a beam with Av < Av_min (No stirrups or not enough stirrups).
... but h <= 10 inches, so we don't have to meet Av_min requirements per Table 9.6.3.1 to be able to use 22.5.5.1 (c).

No axial load assumed, so no adjustment to Vc was done using Nu.

You should remove some rebar or increase your beam or slab depth (𝜌 is greater than 𝜌_max)!

Hmmm, we're in the transition zone... Check your design. phi = 0.83.



**Make Summary Table**

In [271]:

print("___CALCULATION TABLE___")

value_table_with_formulas = [
    ["fy (ksi)", fy, "ksi", "Given"],
    ["Es (ksi)", Es, "ksi", "Given"],
    ["f'c (ksi)", fc, "ksi", "Given"],
    ["b (in)", b, "inches", "Given"],
    ["d (in)", d, "inches", "Given"],
    ["As (in^2)", As, "sqin", "Given"],
    ["Av (in^2)", Av, "sqin", "Given"],
    ["...","","",""],
    ["Av_min (in^2)", f"{Av_min:.3f}", "sqin", "ACI 318-19 Table 9.6.3.4 for nonprestressed members"],
    ["𝜆s", f"{lambda_size:.3f}", "unitless", "ACI 318-19 22.5.5.1.3"],
    ["et", f"{et:.5f}", "in/in", "from using similar triangle relationship between c/eu = (d-c)/es where es = et"],
    ["ey", f"{ey:.5f}", "in/in", "ey = fy/Es or 0.002 if fy = 60 ksi (21.2.2.1 and 21.2.2.2)"],
    ["𝛽1", f"{beta1:.2f}", "unitless", "ACI 318-19 Table 22.2.2.4.3"],
    ["a (in)", f"{a:.2f}", "inches", "a = As*fy/(0.85*fc*b) based on Whitney stress block distribution and allowance per R22.2.2.3"],
    ["c (in)", f"{c:.2f}", "inches", "c = a/𝛽1"],
    ["𝜌", f"{rho:.4f}",         "sqin/in/in", "𝜌 = As/(b*d)"],
    ["𝜌_max", f"{rho_max:.4f}", "sqin/in/in", "𝜌_max = 0.85*𝛽1 * fc/fy * (.003)/(eu + ey + 0.003) Note: eu = 0.003 if not overridden"],
    ["𝜌_bal", f"{rho_bal:.4f}", "sqin/in/in", "𝜌_bal = 0.85*𝛽1 * fc/fy * (eu)/(eu+ey)"],
    ["𝜌_min", f"{rho_min:.4f}", "sqin/in/in", "𝜌_min = 3*sqrt(fc*1000)/fy/1000 and less than 200/(fy*1000)"],
    ["...","","",""],
    ["𝜙", f"{phi:.2f}",   "factor", "Moment material reduction factor based on rho, rho_bal, rho_max,\n  and/or, if in transition zone, based on Table 21.2.2 (d)"],
    ["Mn (kip-in)", f"{Mn:.2f}",   "kip-in", "Mn = As*fy*(d-a/2)"],
    ["Vc (kip)", f"{Vc:.2f}",  "kip", "ACI 318-19 7.6.3.1 or 9.6.3 and 22.5.5 (assuming no axial load)"],
    ["...","","",""],
    ["𝜙Mn (kip-in)", f"{phi_Mn:.2f}", "kip-in", f"𝜙Mn = {phi:.2f}*{Mn:.2f} kip-in"],
    ["𝜙Mn (kip-ft)", f"{phi_Mn/12:.2f}",   "kip-ft", "𝜙Mn shown in kip-ft"],
    ["𝜙Vc (kip)", f"{phi_Vc:.2f}",   "kip", "𝜙Vc = 0.75*Vc"]
]

headers = ["Parameter", "Value", "Units", "Formula/Notes"]

print(tabulate(value_table_with_formulas, headers, tablefmt="simple", numalign="right"))

___CALCULATION TABLE___
Parameter      Value    Units       Formula/Notes
-------------  -------  ----------  --------------------------------------------------------------------------------------------
fy (ksi)       60       ksi         Given
Es (ksi)       29000    ksi         Given
f'c (ksi)      5        ksi         Given
b (in)         12       inches      Given
d (in)         6.19     inches      Given
As (in^2)      1.76     sqin        Given
Av (in^2)      0        sqin        Given
...
Av_min (in^2)  0.010    sqin        ACI 318-19 Table 9.6.3.4 for nonprestressed members
𝜆s             1.000    unitless    ACI 318-19 22.5.5.1.3
et             0.00417  in/in       from using similar triangle relationship between c/eu = (d-c)/es where es = et
ey             0.00200  in/in       ey = fy/Es or 0.002 if fy = 60 ksi (21.2.2.1 and 21.2.2.2)
𝛽1             0.80     unitless    ACI 318-19 Table 22.2.2.4.3
a (in)         2.07     inches      a = As*fy/(0.85*fc*b) based on Whitney stre